In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import auc

from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, accuracy_score

from scipy.special import softmax
from scipy.optimize import differential_evolution
from scipy.spatial import distance_matrix


import seaborn as sns

import faiss

# Leer datos

In [ ]:
df = pd.read_csv('../src/data/train.txt',sep='|',index_col='ID')

In [ ]:
df_test = pd.read_csv('../src/data/test.txt',sep='|',index_col='ID')
df_test['CLASE'] = "NONE"

In [ ]:
df_join = pd.concat([df,df_test],axis=0)

## Preprocesar NA

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder().fit(df.CLASE)

df_final = df_join.copy()
df_final.MAXBUILDINGFLOOR = df_final.MAXBUILDINGFLOOR.fillna(1)
df_final.CADASTRALQUALITYID = df_final.CADASTRALQUALITYID.fillna('7')
df_final.CADASTRALQUALITYID = df_final.CADASTRALQUALITYID.map({'9': '0','8': '1',
                                                                 '7': '2',
                                                                 '6': '3',
                                                                 '5': '4',
                                                                 '4': '5',
                                                                 '3': '6',
                                                                 '2': '7',
                                                                 '1': '8',
                                                                 'C': '9',
                                                                 'B': '10',
                                                                 'A': '11', }).astype(np.int)

## Transform RGBN to prob

In [ ]:
#
# Transform RGBN to prob
#

x = df_join[[x for x in df_join.columns if "Q_" in x]]
y = df_join.CLASE
clase = df.CLASE

x_uniq = x.drop_duplicates().values
x = x.values
probs = np.zeros((x_uniq.shape[0],le.classes_.size))
for i in range(x_uniq.shape[0]):
    d = dict(zip(*np.unique(y[(x == x_uniq[i]).all(1)],return_counts=True)))
    v = np.array([d.get(x,0) for x in le.classes_])
    v = v/v.sum()
    probs[i] = v
probs[np.isnan(probs)] = 0.

# Replace single instances with most common
single_inst = np.isin(probs.max(1),[0,1])
default_label = probs[single_inst].sum(0).argmax()
default_v = np.zeros(probs.shape[1])
default_v[default_label] = 1
probs[single_inst] = default_v

# Generate x_prob
x_prob = np.zeros((x.shape[0],le.classes_.size))
for r,v in zip(x_uniq,probs):
    x_prob[(r == x).all(1)] = v
    
rgbn_df = pd.DataFrame(
    x_prob,
    index=df_join.index,
    columns=[f'RGBN_PROB_{c}' for c in le.classes_])

df_final = pd.concat([df_final,rgbn_df],axis=1)

In [ ]:
from sklearn.metrics import roc_curve
# yp = le.inverse_transform(x_prob.argmax(1)[:clase.size])

for i,c in enumerate(le.classes_):
    a,b,_ = roc_curve(df.CLASE == c,x_prob[:clase.size,i])
    plt.plot(a,b)
plt.gca().set_aspect(aspect='equal')
plt.ylabel('TPR')
plt.xlabel('FPR')
plt.legend(le.classes_)
# x_prob

## Transformar coordenadas reales

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

df_xy = df_final[['X','Y','CLASE']]


xy = StandardScaler().fit_transform(df_xy[['X','Y']])
df_xy.loc[:,['X','Y']] = xy

# clase = LabelEncoder().fit_transform(df_join.CLASE).ravel()

l= np.array([0.9081377,0.5703538,40.455716,-3.627461,0.9281298,0.5705203,40.455762,-3.62622,0.9360858,0.5848481,40.456461,-3.625699,0.7416685,0.7010128,40.46215,-3.638498,0.7613676,0.9099505,40.472564,-3.637233,0.7924592,0.7564925,40.464968,-3.635146,0.2415585,0.07398904,40.430733,-3.670671,0.2475881,0.0706299,40.430648,-3.670359,0.2370961,0.003628214,40.42735,-3.670965,0.3055157,-0.987926,40.37801,-3.665981,0.3063767,-1.012424,40.376827,-3.665957,0.1932484,-0.9052158,40.382082,-3.673394,-0.2123642,0.1171437,40.432837,-3.700311,-0.237763,0.03549111,40.428676,-3.701912,-0.2260408,-0.008040646,40.426585,-3.701115,-2.222735,-0.7149885,40.390552,-3.830743,-2.452621,-0.6939522,40.391495,-3.845708,-2.524583,-0.6638325,40.392986,-3.850581,-3.078021,2.979496,40.573765,-3.888845,-3.125023,2.962788,40.572925,-3.891917,-3.158685,3.144402,40.581957,-3.894241,0.09576253,3.722043,40.612135,-3.682071,0.0699344,3.720931,40.61206,-3.683762,-0.02520764,3.612812,40.606622,-3.689842])
pxs,pys,mys,mxs = l.reshape(-1,4).T

from sklearn.linear_model import LinearRegression
lmx = LinearRegression()
lmx.fit(np.array(pxs)[:,None],mxs)
lmy = LinearRegression()
lmy.fit(np.array(pys)[:,None],mys)

df_xyt = df_xy.copy()
df_xyt.X = lmx.predict(df_xy.X.to_numpy()[:,None])
df_xyt.Y = lmy.predict(df_xy.Y.to_numpy()[:,None])

In [ ]:
fig = px.scatter_mapbox(df_xyt, lat="Y", lon="X",color="CLASE", zoom=10, height=800, width=1200)
fig.update_traces(marker=dict(size=8),
                  selector=dict(mode='markers'))
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Geom transformation

In [ ]:
ss = StandardScaler()
geom = df_join[[x for x in df_join.columns if "GEOM" in x]].copy()
geom.iloc[:,1:] = np.log(geom.iloc[:,1:])
geom = ss.fit_transform(geom)
for d in (geom.T):
    sns.distplot(d)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
knn = KNeighborsClassifier(4, n_jobs=-1, metric='manhattan', weights='distance')
clase = df.CLASE

from sklearn.model_selection import cross_val_predict
clase_pred = cross_val_predict(knn,geom[:clase.size],clase,method='predict_proba')

for i,c in enumerate(le.classes_):
    a,b,_ = roc_curve(df.CLASE == c,clase_pred[:,i])
    plt.plot(a,b)
plt.gca().set_aspect(aspect='equal')
plt.ylabel('TPR')
plt.xlabel('FPR')
plt.legend(le.classes_)

In [ ]:
fig,ax=plt.subplots(1)
for c in le.classes_:
    sns.distplot(geom[df_join.CLASE == c,0],ax=ax)
plt.legend(le.classes_)

fig,ax=plt.subplots(1)
for c in le.classes_:
    sns.distplot(geom[df_join.CLASE == c,1],ax=ax)

fig,ax=plt.subplots(1)
for c in le.classes_:
    sns.distplot(geom[df_join.CLASE == c,2],ax=ax)
    
fig,ax=plt.subplots(1)
for c in le.classes_:
    sns.distplot(geom[df_join.CLASE == c,3],ax=ax)

In [ ]:
geom = df_join[[x for x in df_join.columns if "GEOM" in x]].copy()
geom.iloc[:,1:] = np.log(geom.iloc[:,1:])
geom = ss.fit_transform(geom)
for d in (geom.T):
    sns.distplot(d)
    
nlist = 100
k = 4
agg = np.mean

res = []

db_matrix = geom

for c in le.classes_:
    class_mask = df_join.CLASE == c
#     print(class_mask.sum())
    
    mean_d = np.zeros((class_mask.size,k))
    
    db = db_matrix[class_mask].astype(np.float32)
    rest = db_matrix[~class_mask].astype(np.float32)

    # Train db           # add may be a bit slower as well
    nn = NearestNeighbors(k+1,n_jobs=-1,metric='manhattan')
    nn.fit(db)
    
    # Predict train
    D,I = nn.kneighbors(db)
    mean_d[class_mask] = (D[:,1:])
    
    # Predict rest
    D, I = nn.kneighbors(rest)
    mean_d[~class_mask] = (D[:,:-1])
    
    res.append(mean_d)
res = np.array(res)
res = agg(res,axis=2).T

#
# GEOM DIST DATAFRAME
#
geom_dist = pd.DataFrame(res,index=df_join.index,columns=[f'GEOM_DIST4_{c}' for c in le.classes_])
df_final = pd.concat([df_final,geom_dist],axis=1)

#
# GEOM DIST SOFTMAX
#
res_log = np.log(res+3e-2)
res_log -= res_log.min()
res_log /= res_log.max()
res_log = 1-res_log

geom_nn4_prob = pd.DataFrame(softmax(res_log,axis=1),index=df_join.index,columns=[f'GEOM_PROB_{c}' for c in le.classes_])
df_final = pd.concat([df_final,geom_nn4_prob],axis=1)

In [ ]:
from sklearn.metrics import roc_curve
from scipy.special import softmax

for i,c in enumerate(le.classes_):
    a,b,_ = roc_curve(df.CLASE == c,softmax(res_log,axis=1)[:clase.size,i])
    plt.plot(a,b)
plt.gca().set_aspect(aspect='equal')
plt.ylabel('TPR')
plt.xlabel('FPR')
plt.legend(le.classes_)
# x_prob

# Distance transform

In [ ]:
df_join[['X','Y']]

In [ ]:
nlist = 100
k = 5
agg = np.mean

res = []

db_matrix = np.ascontiguousarray(df_xyt[['X','Y']].values.astype(np.float32))
# db_matrix = db_matrix-db_matrix.min(0)

d = db_matrix.shape[1]                           # dimension
nlist = 100

global_quantizer = faiss.IndexFlatL2(d)  # the other index    
global_index = faiss.IndexIVFFlat(global_quantizer, d, nlist)

global_index.train(db_matrix)
global_index.add(db_matrix)    

D,I = global_index.search(db_matrix,k+1)
res_global = D[:,1:]
res_global[(res_global[:,:] == 0)] = 1e-15

for c in le.classes_:
    class_mask = df_join.CLASE == c
    
    mean_d = np.zeros((class_mask.size,k))
    
    db = db_matrix[class_mask]
    rest = db_matrix[~class_mask]


    # Train db
    quantizer = faiss.IndexFlatL2(d)  # the other index    
    index = faiss.IndexIVFFlat(quantizer, d, nlist)
#     index = faiss.IndexFlatL2(d)
    index.train(db)
    index.add(db)                  # add may be a bit slower as well
    
    # Predict train
    D, I = index.search(db, k+1)     # neighbors of the 5 last queries
#     mean_d[class_mask] = agg(D[:,1:],axis=1)
    mean_d[class_mask] = (D[:,1:])
    
    # Predict rest
    D, I = index.search(rest, k)     # neighbors of the 5 last queries
#     mean_d[~class_mask] = agg(D,axis=1)
    mean_d[~class_mask] = (D)
    
    res.append(mean_d)
res = np.array(res).transpose(1,0,2)
res[res == 0] = 1e-15
# res = agg(res,axis=2).T

In [ ]:
res_pond = (res/np.mean(res_global[:,None,:],axis=2,keepdims=True))
res_pond = res_pond.reshape(-1,k*7)
# res_pond = ((res_pond)-res_pond.mean())/res_pond.std()

In [ ]:
xy_dens_dist = pd.DataFrame(res_pond,index=df_join.index,columns=[f'XY_DENS_{c}_{i}' for c in le.classes_ for i in range(k)])
df_final = pd.concat([df_final,xy_dens_dist],axis=1)

# Prediction 

In [ ]:
basic_cols = ['CONTRUCTIONYEAR', 'MAXBUILDINGFLOOR', 'CADASTRALQUALITYID','AREA']
rgbn_cols = [x for x in df_final.columns if "RGBN_PROB_" in x]
geom_ori_cols = [x for x in df_final.columns if "GEOM_R" in x]
geom_dist_cols = [x for x in df_final.columns if "GEOM_DIST4" in x]
geom_prob_cols = [x for x in df_final.columns if "GEOM_PROB" in x]
xy_dens_cols = [x for x in df_final.columns if "XY_DENS" in x]
xy_ori_cols = ['X','Y']

In [ ]:
clase_weights = np.array([6.26193840e-05, 4.64739874e-05, 4.55873618e-05, 3.83198034e-05, 3.78393795e-05, 4.81255214e-06, 4.26270960e-05])
sample_weight = clase_weights[le.fit_transform(df.CLASE)].ravel()

In [ ]:
ss = StandardScaler().fit(df_join[['X','Y']])
xy_temp = ss.transform(df_join[['X','Y']])
xy_temp = np.array([lmx.predict(xy_temp[:,:1]),lmy.predict(xy_temp[:,1:])]).T

points = np.array([(2.207524e9, 165.5605e6), (2.207524e9, 165.5605e6), (2.170449e9, 166.0036e6),
          (2.205824e9, 166.3199e6), (2.250042e9, 166.2673e6), (2.270527e9, 165.9025e6),
          (2.274459e9, 165.5947e6), (2.269886e9, 165.3261e6), (2.211719e9, 165.1699e6),
          (2.156419e9, 165.2959e6), (2.142472e9, 165.4747e6), (2.141374e9, 165.8068e6),
          (2.166906e9, 165.7316e6), (2.187454e9, 165.4168e6), (2.174702e9, 165.481e6),
          (2.202014e9, 165.5483e6), (2.215004e9, 165.4046e6), (2.196768e9, 165.4717e6),
          (2.236186e9, 165.4013e6), (2.220204e9, 165.4714e6), (2.219742e9, 165.8038e6)])
points = ss.transform(points)
points = np.array([lmx.predict(points[:,:1]),lmy.predict(points[:,1:])]).T

distances = np.array([np.linalg.norm(xy_temp - b, axis=1) for b in points])
_= plt.boxplot(np.array(distances).T)

In [ ]:
distances_cols = [f'C_DIST_{i}' for i in range(points.shape[0])]
distances_df = pd.DataFrame(distances.T,columns=distances_cols,index=df_join.index)
df_final = pd.concat([df_final,distances_df],axis=1)

df_final[xy_dens_cols] = np.clip(df_final[xy_dens_cols],-np.inf,np.exp(20))

In [ ]:

# define N points P
N = 7

yl = le.transform(df.CLASE)
yl_hist = (clase_weights*(np.unique(df.CLASE,return_counts=True)[1])**2)
def get_hist(mini_yl):
    mini_yl_hist = np.histogram(mini_yl,bins=le.classes_.size, range=(0,le.classes_.size), density=False)[0]
    mini_yl_hist = mini_yl_hist/ yl_hist
    mini_yl_hist = mini_yl_hist / mini_yl_hist.sum()
    return mini_yl_hist

xyt = df_xyt[['X','Y']].values[:df.shape[0]]
    
def eval_class_dist(p):    
    p = p.reshape(2,-1).T

    # for x in XY
        # get closer P
        # assign p

    closer_p = np.argmin([np.power(xyt - p[i:i+1],2).sum(1) for i in range(N)],0)
    # strain point
    if np.unique(closer_p).size != N:
        return 0
    
    # for p in P:
        # get normalized distribution
    p_hist = np.array([get_hist(yl[closer_p == i]) for i in range(N)])
    p_size = np.array([np.clip(5*(closer_p==i).sum()/(yl.size/N),0,1) for i in range(N)])
    #     print(p_hist)
    # maximize distance matrix
    # hist_dist = distance_matrix(p_hist,p_hist).sum()
    hist_dist = distance_matrix(p_hist,p_hist)*(p_size[:,None]*p_size[None,:])
    return -hist_dist.sum()

# for i in range(10,100):
#     np.random.seed(i)
#     p = np.random.uniform(xyt.min(0),xyt.max(0),[N,2]).ravel()
#     print(i,eval_class_dist(p))

de_dist = differential_evolution(
    eval_class_dist,
    bounds=list(zip(*np.repeat((xyt.min(0),xyt.max(0)),N,axis=1))),
    workers=-1,
    maxiter=1000,
)
de_best_x = de_dist.x.reshape(2,-1).T

In [ ]:
xyt_all = df_xyt[['X','Y']].values
dist_to_de_points = np.sqrt((((xyt_all[:,None,:]-de_best_x[None,:,:]))**2).sum(-1))
dist_de_cols =[f'DE_DIST_{i}' for i in range(dist_to_de_points.shape[1])]
dtdp_df = pd.DataFrame(dist_to_de_points,columns=dist_de_cols,index=df_join.index)
df_final = pd.concat([df_join,dtdp_df],axis=1)

In [ ]:
p = de_best_x

# for x in XY
    # get closer P
    # assign p
yl = le.transform(df.CLASE)
yl_hist = (clase_weights*(np.unique(df.CLASE,return_counts=True)[1])**2)
def get_hist(mini_yl):
    mini_yl_hist = np.histogram(mini_yl,bins=le.classes_.size, range=(0,le.classes_.size), density=False)[0]
    mini_yl_hist = mini_yl_hist/ yl_hist
    mini_yl_hist = mini_yl_hist / mini_yl_hist.sum()
    return mini_yl_hist

closer_p = np.argmin([np.power(xyt - p[i:i+1],2).sum(1) for i in range(N)],0)

# for p in P:
        # get normalized distribution
p_hist = np.array([get_hist(yl[closer_p == i]) for i in range(N)])
p_size = np.array([np.clip(5*(closer_p==i).sum()/(yl.size/N),0,1) for i in range(N)])
#     print(p_hist)
# maximize distance matrix
# hist_dist = distance_matrix(p_hist,p_hist).sum()
hist_dist = distance_matrix(p_hist,p_hist)*(p_size[:,None]*p_size[None,:])


plt.imshow(hist_dist)

In [ ]:
geom = df_join[[x for x in df_join.columns if "GEOM" in x]].copy()
geom.iloc[:,1:] = np.log(geom.iloc[:,1:])
geom = ss.fit_transform(geom)
for d in (geom.T):
    sns.distplot(d)
    
nlist = 100
k = 4
agg = np.mean

res = []

db_matrix = geom

xyt_all = df_xyt[['X','Y']].values
p = de_best_x
closer_p = np.argmin([np.power(xyt_all - p[i:i+1],2).sum(1) for i in range(N)],0)


for c in le.classes_:
    class_mask = df_join.CLASE == c
    
    mean_d = np.zeros((class_mask.size,k))
    mean_d[:] = np.nan
    mean_check = np.zeros((class_mask.size))
    
    for p_val in np.arange(closer_p.max()):
        
        class_p_mask = class_mask & (closer_p == p_val)
        class_notp_mask = (~class_mask) & (closer_p == p_val)
        
#         print((closer_p == p_val).sum(),class_p_mask.sum(),class_notp_mask.sum())
        
        db = db_matrix[class_p_mask].astype(np.float32)
        rest = db_matrix[class_notp_mask].astype(np.float32)
                
        if db.shape[0] == 0:
            continue
        

        # Train db           # add may be a bit slower as well
        k_max = np.min([k+1,db.shape[0]])
        nn = NearestNeighbors(k_max,n_jobs=-1,metric='manhattan')
        nn.fit(db)

        # Predict train
        D,I = nn.kneighbors(db)
        
        if k_max > 1:
            mean_d[class_p_mask,:k_max-1] = (D[:,1:k_max])
            mean_check[class_p_mask] += 1

        # Predict rest
        D, I = nn.kneighbors(rest)
        mean_d[class_notp_mask,:k_max] = (D[:,:np.min([k,k_max])])
        mean_check[class_notp_mask] += 1

    res.append(mean_d)
    
res = np.array(res)
res[np.isnan(res)] = np.max(res[~np.isnan(res)])*2

res = agg(res,axis=2).T
res = np.log(res+1e-1)

In [ ]:
for i in range(7):
    sns.distplot(res[:,i]+1e-1)

In [ ]:
#
# GEOM DIST DATAFRAME
#
geom_dist_de_cols = [f'GEOM_DIST4_DE_{c}' for c in le.classes_]
geom_dist_de = pd.DataFrame(res,index=df_join.index,columns=geom_dist_de_cols)
df_final = pd.concat([df_final,geom_dist_de],axis=1)

#
# GEOM DIST SOFTMAX
#
# res_log = np.log(res+3e-2)
res_log = res
res_log -= res_log.min()
res_log /= res_log.max()
res_log = 1-res_log

from scipy.special import softmax
geom_nn4_prob_de_cols = [f'GEOM_PROB_DE_{c}' for c in le.classes_]
geom_nn4_prob_de = pd.DataFrame(softmax(res_log,axis=1),index=df_join.index,columns=geom_nn4_prob_de_cols)
df_final = pd.concat([df_final,geom_nn4_prob_de],axis=1)

In [ ]:
# CUSTOM_COLS
df_final['VALUE'] = df_final['AREA'] * df_final['CADASTRALQUALITYID'] * df_final['MAXBUILDINGFLOOR']
df_final['VALUE2'] = df_final['AREA'] * df_final['CADASTRALQUALITYID']
df_final['VALUE3'] = df_final['CADASTRALQUALITYID'] * df_final['MAXBUILDINGFLOOR']
df_final['VALUE4'] = df_final['AREA'] * df_final['MAXBUILDINGFLOOR']
df_final['VALUE5'] = df_final['AREA'] / df_final['MAXBUILDINGFLOOR']+1

custom_cols = ['VALUE', 'VALUE2', 'VALUE3', 'VALUE4', 'VALUE5']

## Salida

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rsearch = RandomizedSearchCV(param_distributions={
    'learning_rate': np.linspace(1e-2,10,100), 
    min_child_weight= np.arange(1,10),
    gamma=np.linspace(0,10,100),
    subsample=np.linspace(0,1,100),
    colsample_bytree=np.linspace(0,1,100),
    max_depth=np.arange(1,50),
    n_estimators=np.arange(100,1500,100)
},n_iter=100)

xgboost = XGBClassifier(n_jobs=-1)
rsearch.fit(xgboost,x,y,fit_params={'sample_weight':sample_weight*y.size})

xgboost = rsearch.best_estimator_

mod_cols = basic_cols+geom_ori_cols+geom_dist_cols+rgbn_cols+xy_dens_cols+custom_cols+dist_de_cols+geom_dist_de_cols+geom_nn4_prob_de_cols

x = df_final[mod_cols].iloc[:df.shape[0]]
y = df.CLASE

xgboost.fit(x,y,sample_weight=sample_weight*y.size)

fig,ax = plt.subplots(1,figsize=(20,20))
plot_importance(xgboost,ax=ax)

In [ ]:
x_ts = df_final[mod_cols].iloc[df.shape[0]:]
yp_ts = xgboost.predict_proba(x_ts)

pd.DataFrame({'CLASE':le.inverse_transform(yp_ts.argmax(1))},index=df_test.index).to_csv('../Universidad de Granada_Code Digger.txt',sep='|',encoding='utf-8')